In [1]:
import os
import random
import math
from sklearn.metrics import mean_squared_error
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sn
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pmdarima as pm 
from sklearn.model_selection import train_test_split
from pmdarima.pipeline import Pipeline

In [2]:
os.getcwd()

'C:\\Users\\niles\\project\\project_work\\notebooks'

In [3]:
# Read data:
#os.chdir("..")
df = pd.read_csv("data.csv")

In [4]:
df.head()

,Time,air_pressure[mmHg],air_temperature[degree celcius],relative_humidity[%],wind_speed[M/S],solar_irridiation[W/m²],total_cloud_cover[from ten],electricity_demand_values[kw],heat_demand_values[kw]
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [5]:
len(df)

70080

In [6]:
df = df.dropna()

In [7]:
len(df)

68792

In [8]:
# Rename Columns:
col_mapping = {"Time": "date",
            "air_pressure[mmHg]": "air_pressure",
            "air_temperature[degree celcius]": "air_temperature",
            "relative_humidity[%]": "relative_humidity",
            "wind_speed[M/S]": "wind_speed",
            "solar_irridiation[W/m²]": "solar_irridiation",
            "total_cloud_cover[from ten]": "total_cloud_cover",
            "electricity_demand_values[kw]": "electricity",
            "heat_demand_values[kw]": "heat"}
df = df.rename(columns = col_mapping)

In [9]:
# Change ts to datetime and set as index:
df["date"] = pd.to_datetime(df["date"],format='%Y-%m-%d')
df.head()

,date,air_pressure,air_temperature,relative_humidity,wind_speed,solar_irridiation,total_cloud_cover,electricity,heat
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [10]:
len(df)

68792

In [11]:
df.head()

,date,air_pressure,air_temperature,relative_humidity,wind_speed,solar_irridiation,total_cloud_cover,electricity,heat
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [12]:
df1 = df[["date",'electricity']]

In [13]:
df1 = df1.set_index("date")
df1.head()

,electricity
date,
2010-12-01 00:00:00,289.567557
2010-12-01 01:00:00,260.168520
2010-12-01 02:00:00,247.273585
2010-12-01 03:00:00,257.955878
2010-12-01 04:00:00,258.255081


In [14]:
x = df1['electricity'].values.reshape(-1, 1) #returns a numpy array
min_max_scaler = StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
df1['electricity'] = x_scaled

In [15]:
df1.head()

,electricity
date,
2010-12-01 00:00:00,-0.437728
2010-12-01 01:00:00,-0.560519
2010-12-01 02:00:00,-0.614377
2010-12-01 03:00:00,-0.569760
2010-12-01 04:00:00,-0.568511


In [16]:
davg_df = df1.groupby(pd.Grouper(freq='D')).mean()
davg_df.head()

,electricity
date,
2010-12-01,-0.159240
2010-12-02,-0.270442
2010-12-03,-0.122860
2010-12-04,0.095403
2010-12-05,0.213953


In [17]:
len(davg_df)

2920

In [18]:
dftrain, dftest = train_test_split(davg_df, test_size=0.01)

In [19]:
len(dftest)

30

In [20]:
dftrain, dfvali = train_test_split(dftrain, test_size=0.2)

In [21]:
len(dftrain)

2312

In [22]:
len(dfvali)

578

In [23]:
# best
random.seed(0)

In [25]:
'''
pipeline = Pipeline([
    ("model", pm.AutoARIMA(test='adf',
                           start_p=3, start_q=3, max_p=5, max_q=5, start_P=0, start_Q=0, max_P=5, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', 
                           suppress_warnings=True, random_state=20, n_fits=100))
])
'''
pipeline = Pipeline([
    ("model",pm.AutoARIMA(test='adf',
                           max_p=3, max_q=3, m=7,
                           start_P=1,start_Q=1, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=False))
])
pipeline.fit(dftrain)

 ARIMA(0,1,0)(0,1,0)[7]             : AIC=7095.479, Time=0.05 sec
 ARIMA(0,1,0)(0,1,1)[7]             : AIC=inf, Time=0.79 sec
 ARIMA(0,1,0)(0,1,2)[7]             : AIC=inf, Time=1.73 sec
 ARIMA(0,1,0)(1,1,0)[7]             : AIC=6408.852, Time=0.12 sec
 ARIMA(0,1,0)(1,1,1)[7]             : AIC=inf, Time=1.52 sec
 ARIMA(0,1,0)(1,1,2)[7]             : AIC=inf, Time=1.81 sec
 ARIMA(0,1,0)(2,1,0)[7]             : AIC=6191.079, Time=0.21 sec
 ARIMA(0,1,0)(2,1,1)[7]             : AIC=inf, Time=2.19 sec
 ARIMA(0,1,0)(2,1,2)[7]             : AIC=inf, Time=2.47 sec
 ARIMA(0,1,1)(0,1,0)[7]             : AIC=inf, Time=0.38 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=inf, Time=2.12 sec
 ARIMA(0,1,1)(0,1,2)[7]             : AIC=inf, Time=1.98 sec
 ARIMA(0,1,1)(1,1,0)[7]             : AIC=inf, Time=0.96 sec
 ARIMA(0,1,1)(1,1,1)[7]             : AIC=inf, Time=1.15 sec
 ARIMA(0,1,1)(1,1,2)[7]             : AIC=inf, Time=2.00 sec
 ARIMA(0,1,1)(2,1,0)[7]             : AIC=inf, Time=3.26 sec
 ARIMA(0,

Pipeline(steps=[('model',
                 AutoARIMA(D=1, d=1, error_action='ignore', m=7, max_p=3,
                           max_q=3, stepwise=False, test='adf', trace=True))])

In [47]:
##### univariate arima model predict electricity_demand_values  
# best
#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=5, start_P=0, start_Q=0, max_P=5, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, random_state=20, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=0, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, random_state=20, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=0, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='bic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=1, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='bic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, n_fits=100)


In [26]:
Sample_predictions= pipeline.predict_in_sample(exogenous=None, return_conf_int=False, alpha=0.05, inverse_transform=True)
Actuals = pd.Series.to_numpy(dftrain['electricity'])
Residuals = Actuals-Sample_predictions

C:\ProgramData\Anaconda3\lib\site-packages\pmdarima\arima\arima.py:692: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "


In [27]:
Residuals

array([ 0.05138618,  0.64879093, -1.12452484, ...,  0.97992427,
       -0.31604177,  0.87151959])

In [42]:
forecast, conf = pipeline.predict(n_periods=578, return_conf_int=True)
forecast

array([-4.06852265e-01, -1.26888254e-01, -5.80735603e-01,  6.03079647e-01,
       -4.69535093e-01, -3.15939303e-01,  4.77078213e-02, -1.71154013e-01,
       -1.81456278e-01, -6.19542263e-01,  3.47588582e-01, -3.44343648e-01,
       -5.30745025e-01,  1.27170577e-01, -1.95578131e-01, -1.72049287e-01,
       -6.07891134e-01,  2.14380022e-01, -3.54866695e-01, -5.34011647e-01,
        6.83312334e-02, -3.13550292e-01, -2.27832906e-01, -6.69823407e-01,
        3.14356358e-01, -4.49859826e-01, -5.34180893e-01,  1.95524026e-02,
       -2.91707328e-01, -2.58132027e-01, -6.96607210e-01,  2.20160122e-01,
       -4.47694430e-01, -5.98516113e-01,  4.52042690e-03, -3.37332410e-01,
       -2.86605562e-01, -7.25663185e-01,  1.89365278e-01, -4.86703060e-01,
       -6.20200681e-01, -3.63724243e-02, -3.78176601e-01, -3.23795275e-01,
       -7.63488859e-01,  1.72498098e-01, -5.26265689e-01, -6.52013312e-01,
       -6.96104221e-02, -4.02471957e-01, -3.55603566e-01, -7.94694543e-01,
        1.27574784e-01, -

In [43]:
Actuals = pd.Series.to_numpy(dfvali['electricity'])
Actuals

array([ 0.01415811, -0.49391515, -0.1555076 ,  1.90306704, -0.51896374,
       -0.3454281 , -0.43709716, -0.36952689,  0.33400157, -0.64066743,
       -0.24160327,  0.61392392, -0.68022692, -0.61618406, -0.47381624,
       -0.02116876,  0.52918626,  0.42042739, -0.19705081,  0.29500246,
       -0.18384362, -0.3604248 ,  0.01350344, -0.4508842 ,  1.09870665,
       -0.65443709,  0.35183041, -0.21500597, -0.20770829,  0.22256201,
       -0.59638125, -0.50075549,  0.05216586, -0.09118283, -0.22535348,
       -0.50992466,  1.17337318, -0.03649887,  0.50773201, -0.26419372,
       -0.49625773, -0.24541617,  0.03858817,  0.37246015,  0.19883546,
        0.41321997, -0.47204687,  0.94608281, -0.49499412, -0.46158932,
       -0.49559772, -0.21424482,  0.40558905, -0.42136195,  0.24915144,
       -0.62234071,  0.56627398, -0.35695551,  0.02632716,  0.31130345,
        0.22317715,  0.34665629,  0.37598347, -0.11022993, -0.65943232,
       -0.10377278,  1.14450196,  0.1880728 , -0.01158381, -0.05

In [44]:
mseVali = mean_squared_error(Actuals, forecast)
mseVali

3.21365440124

In [39]:
forecasttrain, _ = pipeline.predict(n_periods=2312, return_conf_int=True)
forecasttrain

array([ -0.40685227,  -0.12688825,  -0.5807356 , ..., -10.78056685,
       -11.11784402, -11.06781566])

In [40]:
Actualstrain = pd.Series.to_numpy(dftrain['electricity'])
Actualstrain

array([ 0.05138618,  0.70017709, -0.42434817, ...,  0.00720098,
       -0.60120013,  0.25402806])

In [41]:
msetrain = mean_squared_error(Actualstrain, forecasttrain)
msetrain

42.069440869859974

In [45]:
forecasttest, conf = pipeline.predict(n_periods=30, return_conf_int=True)
forecasttest

array([-0.40685227, -0.12688825, -0.5807356 ,  0.60307965, -0.46953509,
       -0.3159393 ,  0.04770782, -0.17115401, -0.18145628, -0.61954226,
        0.34758858, -0.34434365, -0.53074502,  0.12717058, -0.19557813,
       -0.17204929, -0.60789113,  0.21438002, -0.3548667 , -0.53401165,
        0.06833123, -0.31355029, -0.22783291, -0.66982341,  0.31435636,
       -0.44985983, -0.53418089,  0.0195524 , -0.29170733, -0.25813203])

In [46]:
Actualstest = pd.Series.to_numpy(dftest['electricity'])
Actualstest

array([ 1.65016285,  0.24249175, -0.34739269,  0.42666775, -0.22279165,
       -0.56480335, -0.23455027, -0.37762995,  0.02420513, -0.62016832,
        0.26560193,  0.38694051, -0.36748591, -0.50604808, -0.36249302,
       -0.45833457, -0.59882359, -0.46336728, -0.32463615,  0.46309638,
        0.20958649, -0.50579558, -0.1519526 , -0.43097922,  0.41177981,
       -0.53306412, -0.45326051, -0.53906262, -0.58146073, -0.00914812])

In [49]:
msetest = mean_squared_error(Actualstest, forecasttest)
msetest

0.2624971928325078

In [50]:
rmse = math.sqrt(msetest)
print('RMSE: %f' % rmse)

RMSE: 0.512345


In [52]:
mse = mean_squared_error(Actuals, forecast)
mse

8.534984766097647

In [53]:
rmse = math.sqrt(mse)
print('RMSE: %f' % rmse)

RMSE: 2.921470
